In [2]:
import json
import re
from pathlib import Path
#import jq
from langchain_community.document_loaders import JSONLoader
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [3]:
import os
os.environ["OPENAI_API_KEY"] = "voc-8162499801266773377505669655d3c05508.40840521"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [4]:
file_path='./generated_adverts_a.jsonl'

In [5]:
with open(file_path, "r") as file:
    for line in file:
        data_entry = json.loads(line)
        # Process each data_entry as a Python dict
        print(data_entry)

{'Index': '1', 'Neighborhood': 'Maplewood, New Jersey', 'Style': 'Colonial', 'Rooms': '6', 'Bedrooms': '3', 'Bathrooms': '2', 'House Size': '1,500 sqft', 'Price': '$350,000', 'Real Estate Description': "Welcome to your new home in the charming neighborhood of Maplewood. This colonial-style house boasts 3 spacious bedrooms, 2 full bathrooms, and a cozy living room perfect for family gatherings. With a generous 1,500 sqft of living space, this home is perfect for a growing family. Enjoy the convenience of a garage and a beautiful garden for outdoor activities. Don't miss out on this opportunity to own a piece of Maplewood for only $350,000!", 'Neighborhood Description': "Maplewood is a family-friendly neighborhood known for its tree-lined streets, top-rated schools, and close-knit community. With easy access to major highways and public transportation, you'll have the best of both worlds - a peaceful suburban lifestyle with the bustling city just a short commute away."}
{'Index': '2', 'N

In [7]:
loader = JSONLoader(
    file_path=file_path,
    jq_schema='.',
    text_content=False,
    json_lines=True)
docs = loader.load()

In [8]:
print(docs[0])

page_content='{"Index": "1", "Neighborhood": "Maplewood, New Jersey", "Style": "Colonial", "Rooms": "6", "Bedrooms": "3", "Bathrooms": "2", "House Size": "1,500 sqft", "Price": "$350,000", "Real Estate Description": "Welcome to your new home in the charming neighborhood of Maplewood. This colonial-style house boasts 3 spacious bedrooms, 2 full bathrooms, and a cozy living room perfect for family gatherings. With a generous 1,500 sqft of living space, this home is perfect for a growing family. Enjoy the convenience of a garage and a beautiful garden for outdoor activities. Don't miss out on this opportunity to own a piece of Maplewood for only $350,000!", "Neighborhood Description": "Maplewood is a family-friendly neighborhood known for its tree-lined streets, top-rated schools, and close-knit community. With easy access to major highways and public transportation, you'll have the best of both worlds - a peaceful suburban lifestyle with the bustling city just a short commute away."}' me

In [9]:
def json_advert_to_string(doc):
    json_to_str = json.dumps(json.loads(doc.page_content), indent=0)
    json_to_str = json_to_str.replace("\"", "").replace(",\n", "\n")
    json_to_str = json_to_str.replace("{\n", "").replace("\n}", "")
    json_to_str = re.sub(r"Index: \d+\n", "", json_to_str)
    return json_to_str


print(json_advert_to_string(docs[0]))

Neighborhood: Maplewood, New Jersey
Style: Colonial
Rooms: 6
Bedrooms: 3
Bathrooms: 2
House Size: 1,500 sqft
Price: $350,000
Real Estate Description: Welcome to your new home in the charming neighborhood of Maplewood. This colonial-style house boasts 3 spacious bedrooms, 2 full bathrooms, and a cozy living room perfect for family gatherings. With a generous 1,500 sqft of living space, this home is perfect for a growing family. Enjoy the convenience of a garage and a beautiful garden for outdoor activities. Don't miss out on this opportunity to own a piece of Maplewood for only $350,000!
Neighborhood Description: Maplewood is a family-friendly neighborhood known for its tree-lined streets, top-rated schools, and close-knit community. With easy access to major highways and public transportation, you'll have the best of both worlds - a peaceful suburban lifestyle with the bustling city just a short commute away.


In [ ]:
embeddings = OpenAIEmbeddings()
vector_store = Chroma(
    collection_name="adverts",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not neccesary
)